In [ ]:
!pip install transformers datasets torch tensorboard




In [ ]:
# MINIMAL VERSION - GUARANTEED TO WORK
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
import os
os.environ["WANDB_DISABLED"] = "true"
from datasets import load_dataset
import numpy as np

# 1. Load everything
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

# 2. Small dataset for quick training
dataset = load_dataset("imdb")
small_train = dataset['train'].select(range(1000))  # First 1000 samples
small_test = dataset['test'].select(range(200))     # First 200 samples

# 3. Tokenize
def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=128)

tokenized_train = small_train.map(tokenize_function, batched=True)
tokenized_test = small_test.map(tokenize_function, batched=True)

# 4. SIMPLE training arguments (no version conflicts)
training_args = TrainingArguments(
    output_dir="./simple_model",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    save_steps=500,
    report_to="none",
)

# 5. Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
)

print("Training simple model...")
trainer.train()

# 6. Test
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

test_texts = ["good movie", "bad movie", "not bad", "not good"]
for text in test_texts:
    result = classifier(text)
    print(f"'{text}' -> {result[0]}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Training simple model...


Step,Training Loss


Device set to use cuda:0


'good movie' -> {'label': 'POSITIVE', 'score': 0.9996633529663086}
'bad movie' -> {'label': 'NEGATIVE', 'score': 0.9999750852584839}
'not bad' -> {'label': 'POSITIVE', 'score': 0.9899027347564697}
'not good' -> {'label': 'NEGATIVE', 'score': 0.9999929666519165}
